In [59]:
from datetime import datetime

from bs4 import BeautifulSoup
from coala_utils.decorators import generate_ordering

In [60]:
@generate_ordering('timestamp', 'id', 'text', 'user')
class Tweet:
    def __init__(self, user, id, timestamp, fullname, text):
        self.user = user
        self.id = id
        self.timestamp = timestamp
        self.fullname = fullname
        self.text = text

    @classmethod
    def from_soup(cls, tweet):
        return cls(
            user=tweet.find('span', 'username').text[1:],
            id=tweet['data-item-id'],
            timestamp=datetime.utcfromtimestamp(
                int(tweet.find('span', '_timestamp')['data-time'])),
            fullname=tweet.find('strong', 'fullname').text,
            text=tweet.find('p', 'tweet-text').text
            if tweet.find('p', 'tweet-text') else ""
        )

    @classmethod
    def from_html(cls, html):
        soup = BeautifulSoup(html, "lxml")
        tweets = soup.find_all('li', 'js-stream-item')
        if tweets:
            for tweet in tweets:
                try:
                    yield cls.from_soup(tweet)
                except AttributeError:
                    pass  # Incomplete info? Discard!

In [61]:
import json
import logging
import random
import sys
from datetime import timedelta, date
from multiprocessing.pool import Pool

if sys.version_info[0] == 2:
    from urllib2 import urlopen, Request, HTTPError, URLError
else:
    from urllib.request import urlopen, Request, HTTPError, URLError

from fake_useragent import UserAgent

from twitterscraper.tweet import Tweet

In [62]:
import json
import logging
import random
import sys
from datetime import timedelta, date
from multiprocessing.pool import Pool

if sys.version_info[0] == 2:
    from urllib2 import urlopen, Request, HTTPError, URLError
else:
    from urllib.request import urlopen, Request, HTTPError, URLError

from fake_useragent import UserAgent
ua = UserAgent()
HEADERS_LIST = [ua.chrome, ua.google, ua['google chrome'], ua.firefox, ua.ff]

INIT_URL = "https://twitter.com/search?f=tweets&vertical=default&q={q}"
RELOAD_URL = "https://twitter.com/i/search/timeline?f=tweets&vertical=" \
             "default&include_available_features=1&include_entities=1&" \
             "reset_error_state=false&src=typd&max_position={pos}&q={q}"


def query_single_page(url, html_response=True, retry=3):
    """
    Returns tweets from the given URL.
    :param url: The URL to get the tweets from
    :param html_response: False, if the HTML is embedded in a JSON
    :param retry: Number of retries if something goes wrong.
    :return: The list of tweets, the pos argument for getting the next page.
    """
    headers = {'User-Agent': random.choice(HEADERS_LIST)}
    req = Request(url, headers=headers)
    try:
        response = urlopen(req).read().decode('utf-8')
        if html_response:
            html = response
        else:
            json_resp = json.loads(response)
            html = json_resp['items_html']

        tweets = list(Tweet.from_html(html))

        if not tweets:
            return [], None

        if not html_response:
            return tweets, json_resp['min_position']

        return tweets, "TWEET-{}-{}".format(tweets[-1].id, tweets[0].id)
    except HTTPError as e:
        logging.exception('HTTPError {} while requesting "{}"'.format(
            e.code, url))
    except URLError as e:
        logging.exception('URLError {} while requesting "{}"'.format(
            e.reason, url))

    if retry > 0:
        logging.info("Retrying...")
        return query_single_page(url, html_response, retry-1)

    logging.error("Giving up.")
    return [], None


def query_tweets_once(query, limit=None, num_tweets=0):
    """
    Queries twitter for all the tweets you want! It will load all pages it gets
    from twitter. However, twitter might out of a sudden stop serving new pages,
    in that case, use the `query_tweets` method.
    Note that this function catches the KeyboardInterrupt so it can return
    tweets on incomplete queries if the user decides to abort.
    :param query: Any advanced query you want to do! Compile it at
                  https://twitter.com/search-advanced and just copy the query!
    :param limit: Scraping will be stopped when at least ``limit`` number of
                  items are fetched.
    :param num_tweets: Number of tweets fetched outside this function.
    :return:      A list of twitterscraper.Tweet objects. You will get at least
                  ``limit`` number of items.
    """
    logging.info("Querying {}".format(query))
    query = query.replace(' ', '%20').replace("#", "%23").replace(":", "%3A")
    pos = None
    tweets = []
    try:
        while True:
            new_tweets, pos = query_single_page(
                INIT_URL.format(q=query) if pos is None
                else RELOAD_URL.format(q=query, pos=pos),
                pos is None
            )
            if len(new_tweets) == 0:
                logging.info("Got {} tweets for {}.".format(
                    len(tweets), query))
                return tweets

            logging.info("Got {} tweets ({} new).".format(
                len(tweets) + num_tweets, len(new_tweets)))

            tweets += new_tweets

            if limit is not None and len(tweets) + num_tweets >= limit:
                return tweets
    except KeyboardInterrupt:
        logging.info("Program interrupted by user. Returning tweets gathered "
                     "so far...")
        if not tweets:
            raise
    except BaseException:
        logging.exception("An unknown error occurred! Returning tweets "
                          "gathered so far.")

    return tweets


def eliminate_duplicates(iterable):
    """
    Yields all unique elements of an iterable sorted. Elements are considered
    non unique if the equality comparison to another element is true. (In those
    cases, the set conversion isn't sufficient as it uses identity comparison.)
    """
    class NoElement: pass

    prev_elem = NoElement
    for elem in sorted(iterable):
        if prev_elem is NoElement:
            prev_elem = elem
            yield elem
            continue

        if prev_elem != elem:
            prev_elem = elem
            yield elem


def query_tweets(query, limit=None):
    tweets = []
    iteration = 1

    while limit is None or len(tweets) < limit:
        logging.info("Running iteration no {}, query is {}".format(
            iteration, repr(query)))
        new_tweets = query_tweets_once(query, limit, len(tweets))
        tweets.extend(new_tweets)

        if not new_tweets:
            break

        mindate = min(map(lambda tweet: tweet.timestamp, new_tweets)).date()
        maxdate = max(map(lambda tweet: tweet.timestamp, new_tweets)).date()
        logging.info("Got tweets ranging from {} to {}".format(
            mindate.isoformat(), maxdate.isoformat()))

        # Add a day, twitter only searches until excluding that day and we dont
        # have complete results for that one yet. However, we cannot limit the
        # search to less than one day: if all results are from the same day, we
        # want to continue searching further into the past: either there are no
        # further results or twitter stopped serving them and there's nothing
        # we can do.
        if mindate != maxdate:
            mindate += timedelta(days=1)

        # Twitter will always choose the more restrictive until:
        query += ' until:' + mindate.isoformat()
        iteration += 1

    # Eliminate duplicates
    return list(eliminate_duplicates(tweets))


def query_all_tweets(query):
    """
    Queries *all* tweets in the history of twitter for the given query. This
    will run in parallel for each ~10 days.
    :param query: A twitter advanced search query.
    :return: A list of tweets.
    """
    year = 2006
    month = 3

    limits = []
    while date(year=year, month=month, day=1) < date.today():
        nextmonth = month + 1 if month < 12 else 1
        nextyear = year + 1 if nextmonth == 1 else year

        limits.append(
            (date(year=year, month=month, day=1),
             date(year=year, month=month, day=10))
        )
        limits.append(
            (date(year=year, month=month, day=10),
             date(year=year, month=month, day=20))
        )
        limits.append(
            (date(year=year, month=month, day=20),
             date(year=nextyear, month=nextmonth, day=1))
        )
        year, month = nextyear, nextmonth

    queries = ['{} since:{} until:{}'.format(query, since, until)
               for since, until in reversed(limits)]

    pool = Pool(20)
    all_tweets = []
    try:
        for new_tweets in pool.imap_unordered(query_tweets_once, queries):
            all_tweets.extend(new_tweets)
            logging.info("Got {} tweets ({} new).".format(
                len(all_tweets), len(new_tweets)))
    except KeyboardInterrupt:
        logging.info("Program interrupted by user. Returning all tweets "
                     "gathered so far.")

    return sorted(all_tweets)

In [96]:
user=[]
id = []
timestamp = []
fullname = []
text = []
for tweet in query_all_tweets("#airbnbwhileblack"):
    user.append(tweet.user.encode('utf-8'))
    id.append(tweet.id)
    timestamp.append(tweet.timestamp)
    fullname.append(tweet.fullname.encode('utf-8'))
    text.append(tweet.text.encode('utf-8'))

In [97]:
print(len(user))
print(len(id))
print(len(timestamp))
print(len(fullname))
print(len(text))

7756
7756
7756
7756
7756


In [98]:
import pandas as pd

In [99]:
df=pd.DataFrame(user,columns=['user'])
df['id']=id
df['timestamp']=timestamp
df['fullname']=fullname
df['text']=text

In [100]:
df

,user,id,timestamp,fullname,text
0,TinaLaBang,621699389044584448,2015-07-16 15:14:19,Q.,#AirbnbWhileBlack #Airbnbpic.twitter.com/SvhFy...
1,TinaLaBang,621699448318525441,2015-07-16 15:14:33,Q.,#AirbnbWhileBlack #Airbnbpic.twitter.com/9SGXF...
2,TinaLaBang,621699538395496448,2015-07-16 15:14:55,Q.,#AirbnbWhileBlack #Airbnbpic.twitter.com/jNNGq...
3,TinaLaBang,621699636559015937,2015-07-16 15:15:18,Q.,#AirbnbWhileBlack #Airbnbpic.twitter.com/3pIGz...
4,TinaLaBang,621699709401481217,2015-07-16 15:15:36,Q.,#AirbnbWhileBlack #Airbnbpic.twitter.com/yWcOH...
5,TinaLaBang,621700039560314882,2015-07-16 15:16:54,Q.,These were all inquiries that I sent for dates...
6,thatguydream,621708152753426432,2015-07-16 15:49:09,Rah.,had to create an entirely different profile ju...
7,thatguydream,621711452081139712,2015-07-16 16:02:15,Rah.,#AirbnbWhileBlack having to use your middle na...
8,thatguydream,621712351880830976,2015-07-16 16:05:50,Rah.,#AirbnbWhileBlack #Airbnbhttp://valleywag.gawk...
9,thatguydream,621714229859479552,2015-07-16 16:13:17,Rah.,#AirbnbWhileBlack #Airbnb your request gets de...


In [101]:
df.to_csv('airbnbwhileblack.csv')